## Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import requests
import os
import zipfile
import matplotlib.pyplot as plt
from random import random
from sklearn.model_selection import train_test_split

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov
from pypfopt import objective_functions

## Download CVM files

In [3]:
# set current directory and change to folder "dados_cvm"
current_dir_orig = os.getcwd()
os.chdir(f"{current_dir_orig}/dados_cvm")

In [4]:
def download_files(years, months_2021, months_2022, url_root, url_root2):
    for year in years:
        download = requests.get(url_root + f"inf_diario_fi_{year}.zip")
        open(f"inf_diario_fi_{year}.zip", "wb").write(download.content)

    for month in months_2021:
        download = requests.get(url_root2 + f"inf_diario_fi_{month}.zip")
        open(f"inf_diario_fi_{month}.zip", "wb").write(download.content)

    for month in months_2022:
        download = requests.get(url_root2 + f"inf_diario_fi_{month}.zip")
        open(f"inf_diario_fi_{month}.zip", "wb").write(download.content)        


# years in which the files will be downloaded
years = range(2014,2021)
months_2021 = range(202101, 202113)
months_2022 = range(202201, 202211)
years_hist = list(range(2019, 2023))    # analysis in the years 2019, 2020, 2021, 2022

url_root = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/"
url_root2 = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/"

current_dir = os.getcwd()

# criteria to download only once
if len(os.listdir(current_dir)) == 0:
    download_files(years, months_2021, months_2022, url_root, url_root2)
else:
    pass


## Extract data from downloaded files

In [5]:
file_list = []

for file in os.listdir(current_dir):
    
        file_zip = zipfile.ZipFile(file)
        
        for sheet in file_zip.namelist():
            cotation = pd.read_csv(file_zip.open(sheet), sep=";", usecols=["CNPJ_FUNDO", "DT_COMPTC","VL_QUOTA"])
            file_list.append(cotation)
            
# change to original directory
os.chdir(current_dir_orig)

## Select relevant data

In [6]:
def data_funds(file_list, cnpj, year, cont):
    
    # year of 2022 is not complete, it's a special case
    if year == 2022:
        
        df = pd.concat(file_list[cont*12:], ignore_index=True)
    
    else:
        
        df = pd.concat(file_list[cont*12:(cont+1)*12], ignore_index=True)
    
    
    # filters dataframes by cnpj
    df_filt = df.loc[df["CNPJ_FUNDO"] == cnpj, ["CNPJ_FUNDO","DT_COMPTC","VL_QUOTA"]]

    # create two lists, one with date and one with price
    dates = list(df_filt['DT_COMPTC'])
    prices = list(df_filt['VL_QUOTA'])

    return dates, prices
    

In [7]:
# import spreadsheet with investment funds and cnpj's
funds_cnpj_df = pd.read_excel(r'./planilha_fundos/FIA1.xlsx')

# dictionary with cnpj's as keys and fund names as values
dic = dict(zip(funds_cnpj_df['CNPJ'], funds_cnpj_df['FUNDS']))

cnpj = funds_cnpj_df.loc[0,"CNPJ"]

# dictionary os the years and respective counters
count = [5, 6, 7, 8]
dic_years = dict(zip(years_hist, count))

dict_proj_df = {}

for year,cont in dic_years.items():
    
    date_comp_hist, _ = data_funds(file_list, cnpj, year, cont)
    
    # create dataframe with the column "Date"
    funds_proj_df = pd.DataFrame(date_comp_hist, columns=["Date"])
    
    # fill dataframe with the prices per date
    for cnpj,fund in dic.items():
        
        _, funds_proj_df[fund] = data_funds(file_list, cnpj, year, cont)
    
    dict_proj_df[year] = funds_proj_df
    

In [8]:
# delete dataframe and list to free memory
del file_list

In [9]:
dict_proj_df[2022]

,Date,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
0,2022-01-03,2.087273,2.068272,66.941062,2.691333,856.540991,6.105349,6.804673,327.869650,3579.395108,...,2.408862,2.881972,14.344273,3.737435,2.540742,743.879925,380.304924,5.804226,165.972678,9.977780
1,2022-01-04,2.072558,2.058300,66.579971,2.655324,839.253381,6.049534,6.669758,320.103228,3490.397154,...,2.380544,2.845904,14.161991,3.693023,2.508709,732.682805,379.785324,5.659240,164.334601,9.868012
2,2022-01-05,1.979066,2.004220,64.988863,2.556089,807.005969,5.886945,6.403214,305.981790,3361.771878,...,2.282264,2.726764,13.739627,3.531902,2.421151,704.013692,370.737878,5.449375,159.215887,9.611510
3,2022-01-06,1.977240,2.013456,65.064224,2.582523,811.807702,5.873162,6.446887,307.218413,3366.073627,...,2.285123,2.724859,13.706812,3.539673,2.427166,701.107891,371.920263,5.440748,158.913186,9.621503
4,2022-01-07,2.021530,2.041456,65.476278,2.591696,811.815156,5.876027,6.503774,307.564571,3377.604716,...,2.284111,2.723043,13.730695,3.555338,2.450096,702.719934,375.630374,5.452494,158.670046,9.618561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2022-10-04,2.660155,2.447933,70.790486,2.777882,819.098087,6.200375,7.054847,335.539413,3290.487456,...,2.268286,3.237858,16.495505,4.012233,2.876146,790.322840,411.489072,5.562801,186.492465,12.352106
191,2022-10-05,2.683836,2.466490,71.171714,2.779496,818.924679,6.225530,7.079094,336.112202,3293.131242,...,2.302405,3.266213,16.544227,4.038069,2.876289,793.694232,414.839965,5.551658,187.227322,12.417251
192,2022-10-06,2.718091,2.473287,71.446229,2.816421,824.459743,6.262937,7.156142,339.164151,3328.216469,...,2.331408,3.283270,16.648048,4.049215,2.897057,797.879904,417.394793,5.573521,188.102574,12.538203
193,2022-10-07,2.679508,2.446277,70.792442,2.754369,803.126789,6.206316,7.081880,335.067268,3262.061120,...,2.290367,3.252631,16.509900,4.009748,2.865642,786.004449,415.760540,5.461500,186.704075,12.464343


## Import Neural Networks predicted weights

In [10]:
def test_split(dataset):
    
    # splits the dataset of predicted weights into a smaller portion for testing
    _, test_set = train_test_split(dataset, test_size=.5, random_state=42)
    
    return test_set.reset_index(drop=True)

In [11]:
# drop the column "Date" and set prices as float
dict_proj_df2 = {}
for key in dict_proj_df.keys():
    
    dict_proj_df2[key] = dict_proj_df[key].drop("Date", axis=1).astype(float)

    
# import .csv file with the weights predicted by the NN's
NNs_weights_df =  pd.read_csv(r'./predicted_weights/FIA1_3x20layers_relu_20142022_pred_weights_output.csv')

windows = NNs_weights_df["window"].unique()

list_winyear = list(zip(windows, years_hist))

dict_pred_weights = {}
dict_pred_port = {}

for w,year in list_winyear:
    
    # splits the dataframes by windows
    df_filt = NNs_weights_df[NNs_weights_df["window"] == w]
    
    # split prediction dataset into possible smaller dataset for testing (if the dataset is too large)
    df_split = df_filt   # test_split(df_filt)
    
    # save the portfolios
    dict_pred_port[year] = df_split.drop(df_split.columns[4:], axis=1).reset_index(drop=True)
    
    # drop all columns, except for the weight columns 
    dict_pred_weights[year] = df_split.drop(list(df_split.columns[:4]) + ["window"], axis=1).reset_index(drop=True)
    

In [12]:
dict_pred_port[2022]

,F1,F2,F3,F4
0,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA
1,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atmos Ações FIC FIA
2,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,AZ Quest Small Mid Caps FIC FIA
3,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Bahia AM Smid Caps Valor FIC FIA
4,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,BNP Paribas Small Caps FIA
...,...,...,...,...
35955,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV
35956,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA
35957,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA
35958,Squadra Long Only FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA


In [13]:
dict_pred_weights[2022].shape

(35960, 4)

In [14]:
dict_pred_weights[2022]

,w_F1,w_F2,w_F3,w_F4
0,4.953883e-07,8.252192e-11,0.026805,0.973195
1,2.513542e-06,1.023365e-09,0.947776,0.052222
2,3.792809e-07,8.413029e-11,0.010736,0.989264
3,8.934522e-07,1.010101e-10,0.081959,0.918040
4,5.985274e-07,8.518463e-11,0.509259,0.490740
...,...,...,...,...
35955,7.703106e-07,6.049842e-09,0.810736,0.189263
35956,6.892502e-11,3.121165e-13,0.999993,0.000007
35957,3.212989e-05,2.318169e-06,0.000160,0.999806
35958,5.235630e-08,9.869703e-10,0.993929,0.006071


## Annual returns and profitability

In [15]:
def calculate_returns_df(year, price_year_df):
    
    # calculates the mean historical returns
    return_1Y = mean_historical_return(price_year_df)
    
    # creates a dataframe for the annual returns
    return_proj_df = pd.DataFrame(index=[year], columns=price_year_df.columns)
    
    for col in return_proj_df.columns:
        for idx in return_proj_df.index:
            return_proj_df.loc[idx,col] = return_1Y[col]

    return return_proj_df

In [16]:
dict_funds_yoyret = {}
dict_funds_projret = {}

for year, price_year_df in dict_proj_df2.items():
    
    # calculate the returns
    dict_funds_yoyret[year] = calculate_returns_df(year, price_year_df)

    # creates and fills the dataframe with the profitability
    funds_projret_df = pd.DataFrame()
    
    for fund in price_year_df.columns:
        funds_projret_df[fund] = price_year_df[fund]/price_year_df.loc[0,fund] - 1.0
    
    dict_funds_projret[year] = funds_projret_df


In [17]:
dict_funds_yoyret[2019]

,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,Brasil Capital FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
2019,0.306426,0.284351,0.319582,0.319822,0.508708,0.45667,0.40127,0.547812,0.513069,0.526173,...,0.465086,0.412047,0.546334,0.405845,0.256422,0.395221,0.176341,0.481071,0.521671,0.47044


In [18]:
dict_funds_projret[2019]

,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,Brasil Capital FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.008917,0.004065,0.012670,0.006356,0.009206,0.005339,0.000760,0.005751,0.005353,0.007877,...,0.009996,0.002710,0.006596,0.003622,0.006396,0.017764,0.001357,0.008871,0.011385,0.006589
2,0.011276,-0.002535,0.005564,-0.001359,0.001717,-0.001707,0.003442,0.001539,0.001691,0.003187,...,0.008217,0.002991,0.003397,0.001281,0.005046,0.010483,0.009887,0.005248,0.018023,0.007574
3,-0.000024,-0.007584,0.004290,-0.003066,-0.005176,-0.012747,-0.001586,-0.006514,-0.004596,-0.002562,...,0.007953,-0.002449,-0.005839,-0.001042,0.003657,0.004708,0.010495,0.000252,0.019165,0.003621
4,0.002718,-0.008128,0.005023,-0.000928,-0.002797,-0.018375,-0.003055,-0.004518,-0.002675,-0.003425,...,0.007003,-0.004304,0.001623,-0.000785,0.005986,0.009961,0.011243,0.000342,0.024982,0.003603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.281447,0.286835,0.311223,0.322159,0.508354,0.450049,0.392960,0.527829,0.501246,0.523618,...,0.463112,0.408359,0.527546,0.402632,0.259332,0.406415,0.170753,0.476495,0.497927,0.464723
249,0.304380,0.299198,0.324695,0.337770,0.518699,0.470125,0.411832,0.550800,0.519318,0.533918,...,0.476186,0.422568,0.552073,0.410079,0.269460,0.416223,0.183271,0.489745,0.522552,0.484514
250,0.309326,0.293095,0.317982,0.327176,0.515334,0.467071,0.406552,0.552190,0.516497,0.533342,...,0.470779,0.416880,0.546989,0.407410,0.261692,0.406073,0.179005,0.488587,0.521211,0.477036
251,0.306307,0.284420,0.319728,0.319908,0.509679,0.456785,0.401384,0.547905,0.513149,0.526014,...,0.464790,0.412203,0.546376,0.407380,0.256501,0.396036,0.176420,0.481065,0.521754,0.470650


## Performance evaluation between Neural and Random Sharpe's

In [19]:
def ranweights_array(n):
    
    # array with random weights
    W = np.random.rand(n, 4)
    W = W/np.sum(W, axis=1, keepdims=True)
    
    return W


In [20]:
def calculate_projsharpe(rf_rate, wghts_list, port_projreturn_df, port_yoyret_df):
    
    # number of random portfolios
    n = 1000
    
    
    #---------------------------------------------------- Random Sharpe's -------------------------------------------------#
    
    # risks of the random weights portfolios
    randwgts_array = ranweights_array(n)
    Rcons = np.dot(randwgts_array, (port_projreturn_df.values).T) + 1.0    
    daily_var = np.diff(Rcons, axis=1)/Rcons[:, :-1]
    risk_mk = np.std(daily_var, axis=1) * np.sqrt(252)
    
    # returns of the random weights portfolios    
    dot_prodreturn = np.dot(randwgts_array, (port_yoyret_df.values).T)
    return_mk = dot_prodreturn.T
    
    # Sharpe ratios of the random weights portfolios    
    sharperatio_mk = (return_mk - rf_rate)/risk_mk
    
    # dataframe with Sharpe ratios of the random weights portfolios      
    iteration_port_df = pd.DataFrame(sharperatio_mk.T, columns=['year'])
    
    
    #---------------------------------------------------- Neural Sharpe ---------------------------------------------------#
    
    cons_return_nn = np.dot(np.array(wghts_list), (port_projreturn_df.values).T) + 1.0
    daily_var_nn = np.diff(np.reshape(cons_return_nn, (1, cons_return_nn.shape[0])), axis=1)/cons_return_nn[:-1]    
    risk_nn = np.std(daily_var_nn, axis=1) * np.sqrt(252)
    
    # returns of the neural nets predicted weights portfolios    
    return_nn = np.dot(np.array(wghts_list), port_yoyret_df.values.T)
    
    # Sharpe ratios of the neural nets predicted weights portfolios    
    maxsharpe_nn = float((return_nn - rf_rate)/risk_nn)
    
    
    #------------------------------------- Comparison between Neural Sharpe and Random Sharpe's ---------------------------#
    
    count_under = iteration_port_df['year'][iteration_port_df['year'] < maxsharpe_nn].count()/n
    
    return count_under

In [21]:
# risk free rate (CDI) by periods
rf_proj_dict = {2019: .0596, 2020: .0276, 2021: .0442, 2022: .0935}

count_under_dict = {}

# performance evaluation in each period
for year in years_hist:
    
    # number of portfolios
    row_len = len(dict_pred_weights[year])
    
    port_year_list = dict_pred_port[year].values.tolist()
    weights_year_list = dict_pred_weights[year].values.tolist()
    
    score_under = []
    
    # evaluation in each portfolio per period
    for k in range(row_len):
        
        # portfolio and weights list
        port_list = list(port_year_list[k])
        wghts_list = list(weights_year_list[k])
        
        # profitability for each fund in the portfolio
        port_projreturn_df = pd.DataFrame(columns = port_list)
        
        for fund in port_list:
            port_projreturn_df[fund] = dict_funds_projret[year][fund]
        
        # annual return for each fund in the portfolio
        port_yoyret_df = pd.DataFrame(index = dict_funds_yoyret[year].index,
                                             columns = port_list
                                     )
        
        for fund in port_list:
            port_yoyret_df[fund] = dict_funds_yoyret[year][fund]
            
        
        # performance analysis
        count_under = calculate_projsharpe(rf_proj_dict[year],
                                                wghts_list,
                                                port_projreturn_df,
                                                port_yoyret_df
                                               )
        
        score_under.append(count_under)
    
    count_under_dict[year] = score_under


In [22]:
count_under_df = pd.DataFrame(count_under_dict)
count_under_df

,2019,2020,2021,2022
0,0.000,0.000,0.000,0.000
1,0.000,0.000,0.775,0.051
2,0.885,0.000,0.106,0.000
3,0.000,0.000,0.000,0.000
4,0.000,0.227,0.091,0.000
...,...,...,...,...
35955,0.008,1.000,1.000,0.000
35956,0.258,0.455,0.585,0.000
35957,0.823,1.000,0.000,1.000
35958,0.001,0.988,0.273,0.842


## Download performance

In [23]:
count_under_df.to_csv(r'./predictions_performace/FIA1_3x20layers_relu_20142022_NN_performance.csv',
                    encoding = 'utf-8-sig',
                    index = False)